In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [11]:
data = keras.datasets.imdb
(train_data, train_labels) , (test_data, test_labels) =data.load_data(num_words= 88000)  # we are using here only first 10000 words because for convinence
print(train_data[0]) # this will return a list which consists of integers which we humans cannot understand so we will convert them into words

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Mapping


In [12]:
word_index= data.get_word_index() #this will return a tuple which consists of values
word_index = {k:(v+3) for k,v in word_index.items()} # where k stands for keys and v stands for values
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value,key) for (key,value) in word_index.items()]) # reversing the word

train_data = keras.preprocessing.sequence.pad_sequences(train_data, value =word_index["<PAD>"],padding="post",maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value =word_index["<PAD>"],padding="post",maxlen=250)


def decode_review(text):
  return " ".join([reverse_word_index.get(i, "?") for i in  text])

print(decode_review(test_data[2]))




king irritated jupiter sends them a stork br br delighted with this formidable looking new king who towers above them the frogs welcome him with a delegation of formally dressed dignitaries the mayor steps forward to hand him the key to the commonwealth as newsreel cameras record the event to everyone's horror the stork promptly eats the mayor and then goes on a merry rampage swallowing citizens at random a title card dryly reads news of the king's appetite throughout the kingdom when the now terrified frogs once more beseech jupiter for help he loses his temper and showers their community with lightning bolts the moral of our story delivered by a hapless frog just before he is eaten is let well enough alone br br considering the time period when this startling little film was made and considering the fact that it was made by a russian émigré at the height of that beleaguered country's civil war it would be easy to see this as a parable about those events starewicz may or may not have 

Model



In [13]:
model =keras.Sequential()
model.add(keras.layers.Embedding(88000,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation="relu"))
model.add(keras.layers.Dense(1,activation="sigmoid"))

model.summary()

model.compile(optimizer ="adam" , loss="binary_crossentropy", metrics=["accuracy"])

x_val = train_data[:10000]
x_train =train_data[10000:]

y_val = train_labels[:10000]
y_train =train_labels[10000:]

fitModel = model.fit(x_train,y_train,epochs=40, batch_size=512, validation_data=(x_val,y_val),verbose=1)

results = model.evaluate(test_data,test_labels)

print(results)




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          1408000   
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1408289 (5.37 MB)
Trainable params: 1408289 (5.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
30/30 [==============================] - 2s 53ms/step - loss: 0.6923 - 

Reviews


In [14]:
# test_review =test_data[0]
# predict = model.predict([test_review])
# print("Review :")
# print(decode_review(test_review))
# print("Prediction:" +str(predict[0]))
# print("Actual :" + str(test_labels[0]))


Saving the model


In [15]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
def review_encode(s):
  encoded = [1]

  for word in s:
    if word.lower() in word_index:
      encoded.append(word_index[word.lower()])
    else:
      encoded.append(2)
  return encoded


model = keras.models.load_model("model.h5")


with open("test1.txt" , encoding ="utf-8") as f:
  for line in f.readlines():
    nline = line.replace( ",", "").replace(".","").replace("(","").replace(")","").replace(":","").replace("\"","").strip().split(" ")
    encode= review_encode(nline)
    encode = keras.preprocessing.sequence.pad_sequences(train_data, value =word_index["<PAD>"],padding="post",maxlen=250)
    predict=model.predict(encode)
    print(line)
    print(encode)
    print(predict[0])


782/782 [==============================] - 1s 2ms/step
"Past Lives" is like a Stephen Sondheim song come to life.

[[   1   14   22 ...    0    0    0]
 [   1  194 1153 ...    0    0    0]
 [   1   14   47 ...    0    0    0]
 ...
 [   1   11    6 ...    0    0    0]
 [   1 1446 7079 ...    0    0    0]
 [   1   17    6 ...    0    0    0]]
[0.9990695]
782/782 [==============================] - 2s 2ms/step


[[   1   14   22 ...    0    0    0]
 [   1  194 1153 ...    0    0    0]
 [   1   14   47 ...    0    0    0]
 ...
 [   1   11    6 ...    0    0    0]
 [   1 1446 7079 ...    0    0    0]
 [   1   17    6 ...    0    0    0]]
[0.9990695]
782/782 [==============================] - 2s 2ms/step
You know the song "The Road You Didn't Take" from "Follies?" How it's all about the different doors that exist along the path of life, but you only get to enter one at a time, leaving you to wonder what was behind the other doors. And that one path isn't necessarily better or worse than anoth